In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._


//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//Circe para trabajar con JSON
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._


//Sttp para realizar request
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._
import sttp.client.basicRequest

//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                  



import org.apache.spark.sql._

import org.apache.spark.sql.functions._


//Reducir numero logs

import org.apache.log4j.{Level, Logger}

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._


//Sttp para realizar request

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._

import sttp.client.basicRequest

//For adding extra dependenies

import $ivy.$                               

//Plotly

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [2]:
//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._


Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@6eeb335c
import spark.implicits._


In [3]:
//Crear URL
def createUri(category: String, widget: String, start_date: String, end_date: String, time_trunc: String, lang: String, 
              geo_trunc: String = "", geo_limit: String = "", geo_ids: String = ""): String = {
  val baseUrl = s"https://apidatos.ree.es/$lang/datos/$category/$widget"
  
// Crear un mapa con los parámetros obligatorios y opcionales
  val params = Map(
    "start_date" -> start_date,
    "end_date" -> end_date,
    "time_trunc" -> time_trunc,
    "geo_trunc" -> geo_trunc,
    "geo_limit" -> geo_limit,
    "geo_ids" -> geo_ids
  ).filter { case (_, v) => v.nonEmpty } // Filtrar los parámetros que no están vacíos

//Especifico el orden concreto de los paramtros ya que, Scala map no te garantiza el orden
  val orderedKeys = Seq("start_date", "end_date", "time_trunc", "geo_trunc", "geo_limit", "geo_ids")

// Unir los parámetros en una cadena
  val queryString = orderedKeys.flatMap(key => params.get(key).map(value => s"$key=$value")).mkString("&")
  
  s"$baseUrl?$queryString"
}

//Lanzar llamada y obtener guardar datos
def getDataApi(apiUrl: String): Either[String, String] = {
 
    val newRequest = basicRequest.get(uri"$apiUrl")
    val response = newRequest.send()
  
  response.body match {
    case Right(body) => 
      Right(body)
      
    case Left(error) => 
      Left(s"Error fetching data: $error")
  }
}

// Transformar response a DataFrame
def responseToDF(response: Either[String, String]): DataFrame = {
    response match {
    case Right(body) => 
        val rdd = spark.sparkContext.parallelize(Seq(body))
        spark.read.json(rdd)
    case Left(error) => 
        println(error)
        spark.emptyDataFrame
    }
}


defined function createUri
defined function getDataApi
defined function responseToDF

### Definición metodo de descarga dinámica

Para el método de creación de URIs en este caso hacemos una modificación ya que la poder tener información por horas el rango máximo de respuesta de REDataApi es mensual.

In [4]:
import java.time.format.DateTimeFormatter
import java.time.LocalDate

val category = "mercados"
val widget = "precios-mercados-tiempo-real"
val time_trunc = "hour"
val lang = "es"

val startYear = 2014
val endYear = 2024

def crearRango(primerAño: Int, ultimoAño: Int): Seq[String]  = {
val rango = for {
  year <- primerAño to ultimoAño
  month <- 1 to 12
} yield {
  val init = LocalDate.of(year, month, 1).toString + "T00:00"
  val end = LocalDate.of(year, month, 1).plusMonths(1).minusDays(1).toString + "T23:59"
  (init, end)
}

rango.map {
    case (init, end) => createUri(category, widget, init, end, time_trunc,lang)
    }
}
val urisMercados = crearRango(startYear,endYear)

import java.time.format.DateTimeFormatter

import java.time.LocalDate


category: String = "mercados"
widget: String = "precios-mercados-tiempo-real"
time_trunc: String = "hour"
lang: String = "es"
startYear: Int = 2014
endYear: Int = 2024
defined function crearRango
urisMercados: Seq[String] = Vector(
  "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2014-01-01T00:00&end_date=2014-01-31T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2014-02-01T00:00&end_date=2014-02-28T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2014-03-01T00:00&end_date=2014-03-31T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2014-04-01T00:00&end_date=2014-04-30T23:59&time_trunc=hour",
  "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2014-05-01T00:00&end_date=2

In [5]:
urisMercados.length

res4: Int = 132

In [6]:
val listMercadosJson = urisMercados.map(getDataApi(_))

listMercadosJson: Seq[Either[String, String]] = Vector(
  Right(
    "{\"data\":{\"type\":\"Precios mercado peninsular en tiempo real\",\"id\":\"mer13\",\"attributes\":{\"title\":\"Precios mercado peninsular en tiempo real\",\"last-update\":\"2014-09-21T15:07:13.000+02:00\",\"description\":null},\"meta\":{\"cache-control\":{\"cache\":\"MISS\"}}},\"included\":[{\"type\":\"Precio mercado spot (\\u20ac\\/MWh)\",\"id\":\"600\",\"groupId\":null,\"attributes\":{\"title\":\"Precio mercado spot (\\u20ac\\/MWh)\",\"description\":null,\"color\":\"#df4a32\",\"type\":null,\"magnitude\":\"price\",\"composite\":false,\"last-update\":\"2014-09-21T15:07:13.000+02:00\",\"values\":[{\"value\":20.02,\"percentage\":1,\"datetime\":\"2014-01-01T00:00:00.000+01:00\"},{\"value\":10.34,\"percentage\":1,\"datetime\":\"2014-01-01T01:00:00.000+01:00\"},{\"value\":5.35,\"percentage\":1,\"datetime\":\"2014-01-01T02:00:00.000+01:00\"},{\"value\":5,\"percentage\":1,\"datetime\":\"2014-01-01T03:00:00.000+01:00\"},{\"v

In [7]:
println(spark.sparkContext.uiWebUrl)


Some(http://454c8a3c015e:4040)


In [8]:
// Filtrar solo las respuestas exitosas (Right)
val successfulResponses = listMercadosJson.collect { case Right(body) => body }

successfulResponses: Seq[String] = Vector(
  "{\"data\":{\"type\":\"Precios mercado peninsular en tiempo real\",\"id\":\"mer13\",\"attributes\":{\"title\":\"Precios mercado peninsular en tiempo real\",\"last-update\":\"2014-09-21T15:07:13.000+02:00\",\"description\":null},\"meta\":{\"cache-control\":{\"cache\":\"MISS\"}}},\"included\":[{\"type\":\"Precio mercado spot (\\u20ac\\/MWh)\",\"id\":\"600\",\"groupId\":null,\"attributes\":{\"title\":\"Precio mercado spot (\\u20ac\\/MWh)\",\"description\":null,\"color\":\"#df4a32\",\"type\":null,\"magnitude\":\"price\",\"composite\":false,\"last-update\":\"2014-09-21T15:07:13.000+02:00\",\"values\":[{\"value\":20.02,\"percentage\":1,\"datetime\":\"2014-01-01T00:00:00.000+01:00\"},{\"value\":10.34,\"percentage\":1,\"datetime\":\"2014-01-01T01:00:00.000+01:00\"},{\"value\":5.35,\"percentage\":1,\"datetime\":\"2014-01-01T02:00:00.000+01:00\"},{\"value\":5,\"percentage\":1,\"datetime\":\"2014-01-01T03:00:00.000+01:00\"},{\"value\":0.5,\"percentage\

In [9]:
successfulResponses.length

res8: Int = 130

In [10]:
def transformDataFrame(df: DataFrame): DataFrame = {
    // Definir los tipos de energía de bajas emisiones
    val transformedDF = df
        .withColumn("included", explode($"included"))
        .withColumn("TipoMercado", $"included.type")
        .withColumn("Values", explode($"included.attributes.values"))
        .select(
            $"Values.datetime".cast("timestamp").as("Fecha"),
            $"TipoMercado",
            $"Values.value".as("Valor"),
            $"Values.percentage".as("Porcentaje")
        )
    //Agrupar y pivotar el dataframe
    transformedDF
    /*.groupBy("Fecha")
    .pivot("TipoMercado")
    .agg(
            sum("Valor").as("Valor"),
            sum("Porcentaje").as("Porcentaje")
        )*/

}

defined function transformDataFrame

In [11]:
val listModelsMercados = successfulResponses.map{
    json => responseToDF(Right(json))
}

json at cmd2.sc:44

16 / 16

json at cmd2.sc:44

16 / 16

json at cmd2.sc:44

...

16 / 16

json at cmd2.sc:44

16 / 16

json at cmd2.sc:44 
 (kill)

0 + 12 / 16

In [12]:
val model2024 = listModelsMercados.map(df => transformDataFrame(df))

model2024: Seq[DataFrame] = Vector(
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 more fields],
  [Fecha: timestamp, TipoMercado: string ... 2 mo

In [13]:
val modelFinal2024 = model2024.reduce(_ union _)

modelFinal2024: DataFrame = [Fecha: timestamp, TipoMercado: string ... 2 more fields]

In [14]:
modelFinal2024.cache()

res13: DataFrame = [Fecha: timestamp, TipoMercado: string ... 2 more fields]

In [ ]:
val pivotModel = modelFinal2024
    .groupBy("Fecha")
    .pivot("TipoMercado")
    .agg(
            sum("Valor").as("Valor"), 
            sum("Porcentaje").as("Porcentaje")
        )
    .withColumnRenamed("PVPC (€/MWh)_Valor", "Valor_PVPC")
    .withColumnRenamed("PVPC (€/MWh)_Porcentaje", "Porcentaje_PVPC")
    .withColumnRenamed("Precio mercado spot (€/MWh)_Valor", "Valor_Mercado_Spot")
    .withColumnRenamed("Precio mercado spot (€/MWh)_Porcentaje", "Porcentaje_Mercado_Spot")

pivot at cmd14.sc:3 
 (kill)

109 + 18 / 2080

In [20]:
pivotModel.coalesce(1).write
  .option("header", "true")
  .csv("data/Mercados/datasetMercados20-24")

csv at cmd19.sc:3

928 / 928

csv at cmd19.sc:3

928 / 928

csv at cmd19.sc:3

...

200 / 200

csv at cmd19.sc:3

1 / 1

csv at cmd19.sc:3

1 / 1